In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3' 
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
import gc
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload, _unload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [10]:
##### META PARAMS #####

#Size / depth / length of each individual context/path
CONTEXT_SIZE = 16

#Total number of contexts to sample per bag
BAG_SIZE = 256

#Size of fixed length vector to encode to
OUTPUT_SIZE = 512

# Hard limit to filter bags by. Bags which exceed this size won't even be sampled.
# Set to None to turn off
MAX_BAG_SIZE_FILTER = 2048

#Maximum number of labelled commits to train on
MAX_LABELLED_COMMITS = 512

#Maximum number of unlabelled commits to train on
MAX_UNLABELLED_COMMITS = 512

#Batch size for unsupervised training. Higher = faster/better but more memory required
SIAM_BATCH_SIZE = 64

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload(max_commit_bag_size = MAX_BAG_SIZE_FILTER, max_commits = MAX_LABELLED_COMMITS)

Loading Commit lookup table


Loading commit lookups:   0%|          | 0/1024 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/labelled/01_priority_commit_lookups.pickle


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, balance=True)

Generating apache_positive_pairs X_train and y_train:   0%|          | 0/17 [00:00<?, ?it/s]

Generating apache_negative_pairs X_train and y_train:   0%|          | 0/73 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/1024 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 27
Train set split 0.4444444444444444
Test set size 7
Test set split 0.7142857142857143
Unsupervised Train Size 1024


In [8]:
_unload()

In [ ]:
for encoder in tqdm([5,4,2,3,0,1]):
    try:
        model = CommitDiffModel(unsupervised_data_size = len(X_train_unsupervised), siam_batch_size = SIAM_BATCH_SIZE)
        model.initialize(encoder=encoder)
        model.fit_siam(np.array(X_train_unsupervised), epochs=8, verbose=1)
        model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=4, verbose=1)
        score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
        print("Enocder:", encoder)
        print("Score:", score)
    except Exception as e:
        print(e)
    print("------------------------------------------")
    del model
    gc.collect()

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/8
16/16 [==============================] - 6s 376ms/step - loss: 0.0124
Epoch 2/8
16/16 [==============================] - 6s 379ms/step - loss: -0.0247
Epoch 3/8
16/16 [==============================] - 7s 404ms/step - loss: -0.0747
Epoch 4/8
11/16 [===================>..........] - ETA: 2s - loss: -0.1248